# Testing models

In [58]:
MODEL_TO_LOAD = 'saved/vac05-ntbk_fm_1.pt'

to_test = ['native', 'lnn']
test_manif = {t: f'../data/LogiDataset/splits/for_notebooks/{t}.manifest' for t in to_test}

## Imports

In [59]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from warpctc_pytorch import CTCLoss

from time import time
import inspect, sys, os, json
from os.path import dirname, abspath
sys.path.append(dirname(dirname(abspath(inspect.getfile(inspect.currentframe())))))

from pathlib import Path
from os import makedirs
from collections import OrderedDict

from tqdm import tqdm_notebook as tqdm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from data.data_loader import create_binarizer, get_accents_counts
from utils import count_parameters
from models.modules import MaskConv, SequenceWise, BatchRNN, InferenceBatchSoftmax, Lookahead, \
                    supported_rnns, supported_rnns_inv

from tensorboardX import SummaryWriter

import math

from multitask_loss import MtLoss
from torch.utils.data import DataLoader, Dataset
from decoder import GreedyDecoder, BeamCTCDecoder

%reload_ext autoreload
%autoreload 1
%aimport parameters_logi

param = parameters_logi.get_parameters(dev=False, epochs=2, us_en=False)

## Utils

In [60]:
def val_cnts(list_):
    return pd.Series(list_).value_counts()

def extract_num (s):
    return ''.join([c if c.isdigit() else '' for c in s])

def ids_list(manifest):
    ids = []
    with open(manifest) as f:
        for l in f:
            s = l.split('/')
            ids.append(f'{s[4][:-7]}')
    return ids

def make_accent_dict(manifest_path):
    accent_dict = {}
    class_dict = {}
    with open(manifest_path) as f:
        for l in f:
            wav, txt, acc = l.split(',')
            num = extract_num(wav)
            accent = acc.strip()
            if accent not in class_dict:
                new_key = 0 if (len(class_dict) == 0) else max(class_dict.values()) + 1
                class_dict[accent] = new_key
            accent_dict[num] = class_dict[accent]
    return accent_dict, {v: k for k, v in class_dict.items()}

def tile(a, dim, n_tile):
    init_dim = a.size(dim)
    repeat_idx = [1] * a.dim()
    repeat_idx[dim] = n_tile
    a = a.repeat(*(repeat_idx))
    order_index = torch.LongTensor(np.concatenate([init_dim * np.arange(n_tile) + i for i in range(init_dim)]))
    if a.is_cuda:
        order_index = order_index.cuda()
    return torch.index_select(a, dim, order_index)

## Data

In [61]:
class KaldiDeepspeechDataset(Dataset):
    """Defines an iterator over the dataset. This class is intended to be used with PyTorch DataLoader"""
    
    def __init__(self, data_path, labels, sample_ids, transcripts_path,
                 accent_id_dict,  embeddings_path, ivectors_path=None):
        
        self.data_path = data_path
        self.ivectors_path = ivectors_path
        self.transcripts_path = transcripts_path
        self.embeddings_path = embeddings_path
        self.accent_id_dict = accent_id_dict
        self.labels_map = dict([(labels[i], i) for i in range(len(labels))])
        if isinstance(sample_ids, list):
            self._datafiles = sample_ids
        else:
            with open(sample_ids) as f:
                self._datafiles = [x.strip() for x in f.readlines()]
        
    def __getitem__(self, index):
        file_idx = self._datafiles[index]
            
        with open(os.path.join(self.data_path, file_idx)) as f:
            sample = json.load(f)
        sample = torch.FloatTensor(sample)
        
        target = self.accent_id_dict[extract_num(self._datafiles[index])]

        s_id = file_idx

        transcript_path = f'{self.transcripts_path}{file_idx.split("_")[-1]}.txt'
        transcript = self.parse_transcript(transcript_path)

        embedding = torch.load(f'{self.embeddings_path}{s_id}')

        if self.ivectors_path is None:
            return torch.FloatTensor(sample), target, embedding, transcript
        else:
            with open(os.path.join(self.ivectors_path, self._datafiles[index])) as f:
                ivect = json.load(f)
            return torch.FloatTensor(sample), target, transcript, embedding, torch.FloatTensor(ivect)
        
    def parse_transcript(self, transcript_path):
        with open(transcript_path, 'r', encoding='utf8') as transcript_file:
            transcript = transcript_file.read().replace('\n', '')
        transcript = list(filter(None, [self.labels_map.get(x) for x in list(transcript)]))
        return transcript
                      
    def __len__(self):
        return len(self._datafiles)
    
def collate_fn(batch_tot):
    """This function takes list of samples and assembles a batch. It is intended to used in PyTorch DataLoader."""
    batch = list(zip(*batch_tot))
    ivect = None
    
    if len(batch) == 4:
        input_, acc, emb, trs = batch
    elif len(batch) == 5:
        input_, acc, emb, trs, ivect = batch

    input_lens = torch.tensor([len(r) for r in input_])
    acc = torch.tensor(acc)
    
    input_ = nn.utils.rnn.pad_sequence(input_, batch_first=True)

    target_lens = torch.tensor([len(t) for t in trs])

    if ivect is not None:
        ivect = nn.utils.rnn.pad_sequence(ivect, batch_first=True)
        ivect = tile(ivect, 1, 10)
        ivect = ivect[:, :input_.size(1), :]
        input_ = torch.cat([input_, ivect], dim=2)
    
    __, idx = input_lens.sort(descending=True)
    
    targets = np.array(trs)[idx]
    targets = torch.tensor([t for target in targets for t in target])
    
    input_ = input_[idx]
    input_lens = input_lens[idx].int()
    targets = targets.int()
    target_lens = target_lens[idx].int()
    acc = acc[idx].int()

    emb = torch.cat(emb)
    emb = emb[idx]
    emb = emb.view(emb.size(0), 1, emb.size(1))
    emb = tile(emb, 1, input_.size(1))

    if emb.is_cuda:
        input_ = input_.cuda()
    input_ = torch.cat([input_, emb], dim=2)
    
    return input_, input_lens, targets, target_lens, acc

class KaldiDeepspeechDataLoader(DataLoader):
    def __init__(self, *args, **kwargs):
        """
        Creates a data loader for SpeechDatasets.
        """
        super(KaldiDeepspeechDataLoader, self).__init__(*args, **kwargs)
        self.collate_fn = collate_fn
        


In [62]:
def get_test_loader(test_manif):
    test_dict, __ = make_accent_dict(test_manif)

    test_dataset = KaldiDeepspeechDataset(data_path=param['test_kaldi'],
                                  labels=param['labels'],
                                  sample_ids=ids_list(test_manif), 
                                  transcripts_path=param['test_transcripts'],
                                  embeddings_path=param['test_embeddings_100'],
                                  accent_id_dict=test_dict,
                                  ivectors_path=None)

    test_loader = KaldiDeepspeechDataLoader(test_dataset, 
                                    shuffle=True, 
                                    num_workers=0,
                                    batch_size=param['batch_size'])
    return test_loader
    
    
test_loaders = {k: get_test_loader(v) for k, v in test_manif.items()}

# for data in test_loaders['native']:
#     print(data)
#     break

## Model

In [63]:
class AccentClassifier(nn.Module):
    def __init__(self,
                 input_size,
                 num_classes,
                 rnn_type,
                 hidden_size,
                 bn_size,
                 DEBUG = False,):

        super(AccentClassifier, self).__init__()

        self.hidden_size = hidden_size

        self._DEBUG = DEBUG

        self.rnn = rnn_type(input_size, hidden_size, 2, 
                            bidirectional=True, 
                            batch_first=True)

#         self.rnn = BatchRNN(input_size, 
#                             hidden_size,
#                             rnn_type=rnn_type,bidirectional=True,
#                             batch_norm=True)

        self.bn = nn.Sequential(
            nn.BatchNorm1d(hidden_size * 2),
            nn.Linear(hidden_size * 2, 1024),
            nn.ReLU(),
            nn.BatchNorm1d(1024),
            nn.Linear(1024, bn_size),
            nn.ReLU(),
        )

        self.fc = nn.Sequential(
            nn.BatchNorm1d(bn_size),
            nn.Linear(bn_size, num_classes),
            nn.ReLU(),
        )

        self.softmax = nn.Softmax(dim=1)

    def forward(self, x, lens):
        if self._DEBUG:
            print('input x', x.size())

        x = nn.utils.rnn.pack_padded_sequence(x, lens, batch_first=True)
        x, __ = self.rnn(x)
        x, lens = nn.utils.rnn.pad_packed_sequence(x, batch_first=True)

        if self._DEBUG:
            print('after rnn', x.size())
#        
#         x = x.view(x.size(0), x.size(1), 2, self.hidden_size)

#         if self._DEBUG:
#             print('after view', x.size())

        x = x.mean(dim=1)

        if self._DEBUG:
            print('after mean', x.size())

        x = self.bn(x)
        bn = x

        if self._DEBUG:
            print('after bn', x.size())

        x = self.fc(x)

        if self._DEBUG:
            print('after fc', x.size())

        x = self.softmax(x)

        if self._DEBUG:
            print('after softmax', x.size())
        return x, bn


In [64]:

class Head(nn.Module):
    def __init__(self, 
                rnn_type=nn.LSTM, 
                labels="abc", 
                rnn_hidden_size=768, 
                nb_layers=5, 
                bidirectional=True,
                DEBUG=False):

        super(Head, self).__init__()

        # model metadata needed for serialization/deserialization
        self._DEBUG = DEBUG
        self._hidden_size = rnn_hidden_size
        self._nb_layers = nb_layers
        self._rnn_type = rnn_type
        self._labels = labels
        self._bidirectional = bidirectional

        self.conv = MaskConv(nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=(41, 11), stride=(2, 2), padding=(20, 5)),
            nn.BatchNorm2d(32),
            nn.Hardtanh(0, 20, inplace=True),
            nn.Conv2d(32, 32, kernel_size=(21, 11), stride=(2, 1), padding=(10, 5)),
            nn.BatchNorm2d(32),
            nn.Hardtanh(0, 20, inplace=True)
        ))

        rnn_input_size = 1120

        rnns = []
        rnn = BatchRNN(input_size=rnn_input_size, hidden_size=rnn_hidden_size, rnn_type=rnn_type,
                       bidirectional=bidirectional, batch_norm=False)
        rnns.append(('0', rnn))
        for x in range(nb_layers - 1):
            rnn = BatchRNN(input_size=rnn_hidden_size, hidden_size=rnn_hidden_size, rnn_type=rnn_type,
                           bidirectional=bidirectional)
            rnns.append(('%d' % (x + 1), rnn))
        self.rnns = nn.Sequential(OrderedDict(rnns))


    def forward(self, x, lengths):
        if self._DEBUG:
            print('')
            print('# BEGIN HEAD #')
            print('input', x.size())

        lengths = lengths.cpu().int()
        output_lengths = self.get_seq_lens(lengths)

        x = x.view(x.size(0), 1, x.size(1), x.size(2))
        x = x.transpose(2, 3)
        if self._DEBUG:
            print('after view transpose', x.size())

        x, _ = self.conv(x, output_lengths)
        if self._DEBUG:
            print('after conv', x.size())

        sizes = x.size()
        x = x.view(sizes[0], sizes[1] * sizes[2], sizes[3])  # Collapse feature dimension
        x = x.transpose(1, 2).transpose(0, 1).contiguous()  # TxNxH
        if self._DEBUG:
            print('after view transpose', x.size())

        for rnn in self.rnns:
            x = rnn(x, output_lengths)
        if self._DEBUG:
            print('after rnn', x.size())

        self._DEBUG = False
        return x, output_lengths

    def get_seq_lens(self, input_length):
        """
        Given a 1D Tensor or Variable containing integer sequence lengths, return a 1D tensor or variable
        containing the size sequences that will be output by the network.
        :param input_length: 1D Tensor
        :return: 1D Tensor scaled by model
        """
        seq_len = input_length
        for m in self.conv.modules():
            if type(m) == nn.modules.conv.Conv2d:
                seq_len = ((seq_len + 2 * m.padding[1] - m.dilation[1] * (m.kernel_size[1] - 1) - 1) / m.stride[1] + 1)
        return seq_len.int()

class FC(nn.Module):
    def __init__(self, 
                 rnn_hidden_size=768,
                 labels='abc',
                 DEBUG=False,):

        super(FC, self).__init__()

        self._DEBUG = DEBUG
        self._labels = labels
        num_classes = len(self._labels)

        fully_connected = nn.Sequential(
            nn.BatchNorm1d(rnn_hidden_size),
            nn.Linear(rnn_hidden_size, num_classes, bias=False)
        )
        self.fc = nn.Sequential(
            SequenceWise(fully_connected),
        )
        self.inference_softmax = InferenceBatchSoftmax()


    def forward(self, x):
        if self._DEBUG:
            print('')
            print('# BEGIN FC #')
            print('input', x.size())
        x = self.fc(x)
        if self._DEBUG:
            print('after fc', x.size())

        x = x.transpose(0, 1)
        if self._DEBUG:
            print('after transpose', x.size())
        # identity in training mode, softmax in eval mode
        x = self.inference_softmax(x)
        if self._DEBUG:
            print('after softmax', x.size())

        x = x.transpose(0, 1)
        if self._DEBUG:
            print('after transpose', x.size())

        self._DEBUG = False
        return x


class AccentClassifier(nn.Module):
    def __init__(self,
                 input_size,
                 accents={},
                 DEBUG=False,):

        super(AccentClassifier, self).__init__()

        num_classes = len(accents)
        self._DEBUG = DEBUG

        self.fc = nn.Sequential(
            nn.BatchNorm1d(input_size),
            nn.Linear(input_size, 1024),
            nn.ReLU(),
            nn.BatchNorm1d(1024),
            nn.Linear(1024, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Linear(256, num_classes),
            nn.ReLU(),
        )

        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):     
        if self._DEBUG:
            print('')
            print('# BEGIN Acc #')
            print('input', x.size())

        x = x.mean(dim=0)

        if self._DEBUG:
            print('after mean', x.size())

        x = self.fc(x)

        if self._DEBUG:
            print('after fc', x.size())

        x = self.softmax(x)

        if self._DEBUG:
            print('after softmax', x.size())

        self._DEBUG = False
        return x

class Network(nn.Module):
    def __init__(self, 
                rnn_type=nn.LSTM, 
                labels="abc", 
                accents={},
                rnn_hidden_size=768, 
                nb_layers=5, 
                audio_conf=None,
                bidirectional=True,
                DEBUG=False):

        super(Network, self).__init__()

        self.Head = Head(rnn_type=rnn_type, 
                        labels=labels, 
                        rnn_hidden_size=rnn_hidden_size, 
                        nb_layers=nb_layers, 
                        bidirectional=bidirectional,
                        DEBUG=DEBUG)

        self.FC = FC(rnn_hidden_size=rnn_hidden_size,
                     labels=labels,
                     DEBUG=DEBUG,)

        self.Acc = AccentClassifier(input_size=800,
                                  accents=accents,
                                  DEBUG=DEBUG)

    def forward(self, x, lengths):
        x, out_len = self.Head(x, lengths)
        x_fc = self.FC(x)
        x_acc = self.Acc(x)
        return x_fc, x_acc, out_len

## Init

In [65]:
model = torch.load(MODEL_TO_LOAD)

if param['cuda']:
    model.cuda()

optimizer = torch.optim.Adam(model.parameters(), lr=param['lr'][0])

criterion = CTCLoss()

decoder = BeamCTCDecoder(param['labels'], lm_path=param['lm_path'],
                        alpha=0.8, beta=1.,
                        cutoff_top_n=40, cutoff_prob=1.0,
                        beam_width=100, num_processes=param['num_worker'])

target_decoder = GreedyDecoder(param['labels'])

AttributeError: Can't get attribute 'DeepSpeech' on <module '__main__'>

## Testing

In [49]:
def check_wer(targets, targets_len, out, output_len):
    split_targets = []
    offset = 0
    for size in targets_len:
        split_targets.append(targets[offset:offset + size])
        offset += size
        
    decoded_output, _ = decoder.decode(out.data.transpose(0,1), output_len)
    target_strings = target_decoder.convert_to_strings(split_targets)
    
    if False:
        print('targets', targets)
        print('split_targets', split_targets)
        print('out', out)
        print('output_len', output_len)
        print('decoded', decoded_output)
        print('target', target_strings)
    
    wer, cer = 0, 0
    for x in range(len(target_strings)):
        transcript, reference = decoded_output[x][0], target_strings[x][0]
        wer += decoder.wer(transcript, reference) / float(len(reference.split()))
        #cer += decoder.cer(transcript, reference) / float(len(reference))
    wer /= len(target_strings)
    return wer * 100

def check_acc(targets, out):
    out_arg = np.argmax(out, axis=1)
    diff = torch.eq(out_arg.int(), targets)
    acc = torch.sum(diff).float()
    return acc / len(targets) * 100

def get_loss(inputs, inputs_len, targets, targets_len, target_accents, mix):
    
    inputs = inputs.cuda()
    inputs_len = inputs_len.cuda()
    targets = targets.cuda()
    targets_len = targets_len.cuda()

    # Forward pass
    out_fc, out_acc, output_len = model(inputs, inputs_len)

    out_fc = out_fc.cpu()
    targets = targets.cpu()
    targets_len = targets_len.cpu()
    out_acc = out_acc.cpu()

    if False:
        print('## Outputs train')
        print('out', out.size())
        print('targets', targets.size())
        print('output_len', output_len.size())
        print('targets_len', targets_len.size())

    loss_fc = crit_fc(out_fc, targets, output_len, targets_len)
    loss_acc = crit_acc(out_acc, target_accents.long()) * 3000
    loss = mix * loss_fc + (1 - mix) * loss_acc
    
    total_loss = (loss, loss_fc, loss_acc)
    return total_loss, out_fc, out_acc, output_len

In [50]:
def test(test_loader):
    DEBUG=False
    model.eval()
    epoch_val_losses = []
    epoch_wer = []
    with torch.no_grad():
        for data in tqdm(test_loader, total=len(test_loader)): ## ## 
            inputs, inputs_len, targets, targets_len, target_accents = data

            inputs = inputs.cuda()
            inputs_len = inputs_len.cuda()
            targets = targets.cuda()
            targets_len = targets_len.cuda()
            target_accents = target_accents.cuda()

            out, output_len = model(inputs, inputs_len)

            out = out.cpu()
            targets = targets.cpu()
            targets_len = targets_len.cpu()

            if False:
                print('## Outputs test')
                print('out', out)
                print('targets', targets)
                print('output_len', output_len)
                print('targets_len', targets_len)

            val_loss = criterion(out, targets, output_len, targets_len)

            if DEBUG:
                print('val loss', val_loss)

            epoch_val_losses.append(val_loss)

            wer = check_wer(targets, targets_len, out, output_len)
            epoch_wer.append(wer)

    epoch_val_loss = sum(epoch_val_losses) / len(epoch_val_losses) ##
    epoch_wer = sum(epoch_wer) / len(epoch_wer)
    return epoch_wer, epoch_val_loss.item()

In [51]:
results = {k: test(v) for k, v in test_loaders.items()}

RuntimeError: input.size(-1) must be equal to input_size. Expected 320, got 1120

In [ ]:
for k, (wer, acc) in results.items():
    print(f'{k:>6}: {wer:.2f} wer')

In [116]:
# vac05
for k, (wer, acc) in results.items():
    print(f'{k:>6}: {wer:.2f} wer')

native: 54.78 wer
   lnn: 70.38 wer
